In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import tensorflow as tf
tf.__version__

/home/joseph/anaconda3/envs/idermato/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joseph/anaconda3/envs/idermato/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joseph/anaconda3/envs/idermato/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/joseph/anaconda3/envs/idermato/lib

'1.14.0'

In [2]:
IMG_WIDTH = 300
IMG_HEIGHT = 150
NUM_CHANNELS = 3
TARGET_SIZE = (IMG_WIDTH, IMG_HEIGHT)
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, NUM_CHANNELS)
BATCH_SIZE = 8

In [3]:
data = pd.read_csv("images-metadata.csv", index_col="ID")
# data.head()

In [4]:
# files path
all_images = data["PATH"].values
n_samples = len(all_images)
# labels
labels = data["CLASS"].values

print("Total images: ", n_samples)

Total images:  736


In [5]:
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)

In [6]:
labels = tf.keras.utils.to_categorical(labels)
num_classes = labels.shape[1]

In [7]:
def load_image(path, transpose=True):
    image = Image.open(path)
    image = image.convert('RGB')
    if transpose and (image.height > image.width):
        image = image.transpose(Image.ROTATE_90)
    image = image.resize(TARGET_SIZE)
    image = np.array(image) / 255.0
    image = image.reshape(INPUT_SHAPE)
    return image

In [8]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size=32, target_size=(150, 150)):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.target_size = target_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([
            load_image(file_name)
               for file_name in batch_x]), np.array(batch_y)

In [9]:
train_samples = int(n_samples*.8)

In [10]:
train_generator = DataGenerator(all_images[:train_samples], labels[:train_samples], BATCH_SIZE, TARGET_SIZE)
val_generator = DataGenerator(all_images[train_samples:], labels[train_samples:], BATCH_SIZE, TARGET_SIZE)

In [11]:
# Création d'un modèle séquentiel
model = tf.keras.models.Sequential()

# Ajout de couches convolutionnelles et de de pooling
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=INPUT_SHAPE))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Ajout de couches denses
model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax')) # 5 => nombre de classes

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 74, 128)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 72, 64)       73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 17, 32)        0         
____

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
EPOCHS = 1

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS)

25/74 [=========>....................] - ETA: 6:48 - loss: 2.3402 - acc: 0.0300

/home/joseph/anaconda3/envs/idermato/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


33/74 [============>.................] - ETA: 5:34 - loss: 2.3296 - acc: 0.0644

In [ ]:
EPOCHS = 9

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

fig = plt.figure(figsize=(12, 4))

fig.add_subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='best')

fig.add_subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend(loc='best')

plt.show()